In [1]:
from typing import Dict
from langchain.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.prebuilt import create_react_agent


from utils import get_llm

# ==================================================
# 1. LOAD LLM (OLLAMA)
# ==================================================
llm = get_llm()

# ==================================================
# 2. SYSTEM PROMPT
# ==================================================
SYSTEM_PROMPT = """
You are an intelligent ReAct agent.

You can:
- Solve math problems using tools
- Fetch customer information using tools
- Rewrite emails in a specified tone

Rules:
- Use tools only when necessary
- Do not reveal internal reasoning
- Follow the requested tone strictly
"""

# ==================================================
# 3. SUPPORTED EMAIL TONES
# ==================================================
SUPPORTED_TONES = {
    "formal": "Use a professional and respectful tone.",
    "polite": "Use a courteous and friendly tone.",
    "casual": "Use a relaxed and conversational tone.",
    "apology": "Use an apologetic and empathetic tone.",
    "urgent": "Use a firm and time-sensitive tone."
}

# ==================================================
# 4. CUSTOMER DATA
# ==================================================
CUSTOMER_DATA: Dict[str, Dict[str, str]] = {
    "akhila": {
        "name": "Akhila",
        "email": "akhila@gmail.com",
        "phone": "9876543210",
        "city": "Hyderabad"
    }
}

# ==================================================
# 5. TOOL: CALCULATOR
# ==================================================
@tool
def calculate(expression: str) -> str:
    """Perform a mathematical calculation."""
    try:
        result = eval(expression, {"__builtins__": {}})
        return f"Result: {result}"
    except Exception as e:
        return f"Calculation error: {str(e)}"

# ==================================================
# 6. TOOL: CUSTOMER LOOKUP
# ==================================================
@tool
def get_customer_info(name: str) -> str:
    """Retrieve customer details by name."""
    key = name.lower()
    if key in CUSTOMER_DATA:
        c = CUSTOMER_DATA[key]
        return (
            f"Name: {c['name']}\n"
            f"Email: {c['email']}\n"
            f"Phone: {c['phone']}\n"
            f"City: {c['city']}"
        )
    return "Customer not found."

# ==================================================
# 7. TOOL: EMAIL TONE REWRITE
# ==================================================
@tool
def rewrite_email(email_text: str, tone: str) -> str:
    """Rewrite an email in a specified tone."""
    tone = tone.lower()
    if tone not in SUPPORTED_TONES:
        return f"Unsupported tone. Choose from: {', '.join(SUPPORTED_TONES.keys())}"

    instruction = SUPPORTED_TONES[tone]
    return f"{instruction}\n\nEmail:\n{email_text}"

# ==================================================
# 8. CREATE REACT AGENT
# ==================================================
tools = [calculate, get_customer_info, rewrite_email]

agent = create_react_agent(
    model=llm,
    tools=tools
)

# ==================================================
# 9. RUN AGENT
# ==================================================
def run_agent(user_input: str) -> str:
    result = agent.invoke(
        {
            "messages": [
                SystemMessage(content=SYSTEM_PROMPT),
                HumanMessage(content=user_input)
            ]
        }
    )
    return result["messages"][-1].content

if __name__ == "__main__":
    print("---- Calculator Test ----")
    print(run_agent("What is 45 * 6 + 10?"))

    print("\n---- Customer Lookup Test ----")
    print(run_agent("Get customer info for Akhila"))

    print("\n---- Email Rewrite Test ----")
    print(run_agent(
        "Rewrite this email in a formal tone: please send the files asap"
    ))



C:\Users\karth\AppData\Local\Temp\ipykernel_20028\3026825397.py:101: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


---- Calculator Test ----
The result of 45 * 6 + 10 is 280.

---- Customer Lookup Test ----
Here is the information for Akhila:
- Name: Akhila
- Email: akhila@gmail.com
- Phone: 9876543210
- City: Hyderabad

---- Email Rewrite Test ----
The rewritten email in a formal tone is:

"Please send the files at your earliest convenience."


In [2]:
print(run_agent(
        "Rewrite this email in a formal tone: please confirm leave application"))

Here is the email rewritten in a formal tone:

Dear [Recipient's Name],

Could you please confirm my leave application?

Best regards,  
[Your Name]


In [3]:
print(run_agent(
        "write an email to boss for leave application"))

Sure, let's use a **polite** tone for your leave application email to the boss.

Here’s a draft:

---

Subject: Leave Application

Dear [Boss's Name],

I hope this message finds you well. I am writing to formally request a leave of absence from [start date] to [end date]. During my time off, I will ensure that all ongoing projects are properly handed over and any necessary arrangements are made to minimize disruption.

Thank you for considering my request. Please let me know if there is anything else I can do to facilitate this process.

Best regards,

[Your Name]

---

Please replace the placeholders with your actual details before sending the email. Would you like me to send this exact template or make any adjustments?


In [4]:
print(run_agent(
    "Get customer email for Akhila, "
    "calculate 25 * 4 + 10, "
    "and write this email in a formal tone: please send the documents"
))


Here are the results:

- Akhila's email is akhila@gmail.com.
- The result of the calculation 25 * 4 + 10 is 110.
- Here is your email rewritten in a formal tone: 

"Dear Akhila,

Please send the documents.

Best regards,"

You can use this formal version to communicate with Akhila.


In [5]:
print(run_agent(
    "Get customer email for Akhila, "
    "calculate 25 * 4 + 10, "
    "and write this email in a formal tone: regarding above details"
))


Here are the results:

- Akhila's email is akhila@gmail.com.
- The calculation result of \(25 \times 4 + 10\) is 110.
- The rewritten email in a formal tone regarding the above details is as follows:

---

Dear Akhila,

Regarding the above details, please find the required information below.

Best regards,
[Your Name]


In [6]:
print(run_agent(
    "Get customer  name, "
    "calculate 25 * 4, "
    "and write a email for leave application in polite form "
))


Customer information not found. The calculation result is 100. Here is the leave application email written in a polite tone:

Email:
Dear Sir/Madam,

I am writing to apply for a leave of absence. I would like to take a few days off starting from next Monday, the 10th of October.

Thank you for your consideration.
Sincerely,
[Your Name]
